<a href="https://colab.research.google.com/github/AlperSrgn/WebScraping/blob/main/Web_Scraping_from_TrustPilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from time import sleep
import requests
from google.colab import files
import pandas as pd
from bs4 import BeautifulSoup

def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr] if val.has_attr(attr) else None)
    else:
        for val in src:
            list_.append(val.get_text() if val else None)

users = []
userReviewNum = []
ratings = []
locations = []
dates = []
reviews = []

from_page = 1
to_page = 40
company = 'www.apple.com'

for i in range(from_page, to_page+1):
    result = requests.get(fr"https://www.trustpilot.com/review/{company}?page={i}")
    soup = BeautifulSoup(result.content, 'html.parser')

    # Trust Pilot was set up in a way that's not friendly to scraping, so this hacky method will do.
    soup2list(soup.find_all('span', {'class','typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}), users)
    soup2list(soup.find_all('div', {'class','typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua'}), locations)
    soup2list(soup.find_all('span', {'class','typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l'}), userReviewNum)
    soup2list(soup.find_all('div', {'class','styles_reviewHeader__iU9Px'}), dates)
    soup2list(soup.find_all('div', {'class','styles_reviewHeader__iU9Px'}), ratings, attr='data-service-review-rating')
    soup2list(soup.find_all('div', {'class','styles_reviewContent__0Q2Tg'}), reviews)

    # To avoid throttling
    sleep(1)

# Check for length consistency between lists
max_len = max(len(users), len(userReviewNum), len(locations), len(dates), len(reviews), len(ratings))
print(f"Max length: {max_len}")

# Padding shorter lists with None to ensure all lists are of the same length
while len(users) < max_len:
    users.append(None)
while len(userReviewNum) < max_len:
    userReviewNum.append(None)
while len(locations) < max_len:
    locations.append(None)
while len(dates) < max_len:
    dates.append(None)
while len(reviews) < max_len:
    reviews.append(None)
while len(ratings) < max_len:
    ratings.append(None)

# Create the DataFrame
review_data = pd.DataFrame({
    'Username': users,
    'Total reviews': userReviewNum,
    'location': locations,
    'date': dates,
    'content': reviews,
    'Rating': ratings
})

# Show the first few rows
print(review_data.head())


Max length: 920
              Username Total reviews location                date  \
0         Nishit patel   35K reviews       GB        20 hours ago   
1             Aireshan  9.4K reviews       US          3 days ago   
2             Martin M  2.1K reviews       DK          6 days ago   
3  Honest News Network     3 reviews       US  Updated 2 days ago   
4  Christopher Ormonde      1 review       GB          2 days ago   

                                             content Rating  
0  Fake iPhone I was order iPhone16 pro max(256gb...      1  
1  Would rate negative if I couldWould rate negat...      1  
2  Get it FIXUnbelievable that you pay so much mo...      2  
3  Apple Is A Rip OffI asked Google if Mac Mini M...      1  
4  Why is apple music so bad?You cannot use conte...      1  


In [12]:
review_data

,Username,Total reviews,location,date,content,Rating
0,Nishit patel,35K reviews,GB,20 hours ago,Fake iPhone I was order iPhone16 pro max(256gb...,1
1,Aireshan,9.4K reviews,US,3 days ago,Would rate negative if I couldWould rate negat...,1
2,Martin M,2.1K reviews,DK,6 days ago,Get it FIXUnbelievable that you pay so much mo...,2
3,Honest News Network,3 reviews,US,Updated 2 days ago,Apple Is A Rip OffI asked Google if Mac Mini M...,1
4,Christopher Ormonde,1 review,GB,2 days ago,Why is apple music so bad?You cannot use conte...,1
...,...,...,...,...,...,...
915,None,20 reviews,None,None,None,None
916,None,4 reviews,None,None,None,None
917,None,8 reviews,None,None,None,None
918,None,4 reviews,None,None,None,None


In [13]:
# Convert the DataFrame to a CSV file and download it
review_data.to_csv('reviews.csv', index=False)
files.download('reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>